# KI-Training für Anfänger: Blumen-Klassifikation

Dieses Notebook zeigt, wie eine einfache KI trainiert wird.

## 1. Einführung
In diesem Notebook lernen wir:
- Wie man Daten für KI vorbereitet
- Wie man ein einfaches neuronales Netz erstellt
- Wie man die KI trainiert
- Wie man die Genauigkeit testet

Wir verwenden den bekannten **Iris-Datensatz** mit 3 Blumenarten.

## 2. Bibliotheken installieren (nur für JupyterLite nötig)

In [ ]:
# In JupyterLite müssen wir Pakete mit piplite installieren
import sys
if 'pyodide' in sys.modules:
    import piplite
    await piplite.install(['numpy', 'pandas', 'matplotlib', 'scikit-learn'])
    print("✓ Pakete für JupyterLite installiert!")

## 3. Bibliotheken importieren

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

print("✓ Alle Bibliotheken erfolgreich importiert!")

## 4. Daten laden und erkunden

In [ ]:
# Iris-Datensatz laden
iris = load_iris()
X = iris.data  # Features: Kelchblatt- und Blütenblatt-Maße
y = iris.target  # Zielwerte: Blumenart (0, 1, oder 2)

# Als DataFrame anzeigen für bessere Lesbarkeit
df = pd.DataFrame(X, columns=iris.feature_names)
df['Blumenart'] = pd.Categorical.from_codes(y, iris.target_names)

print("📊 Erste 5 Zeilen unserer Daten:")
print(df.head())
print(f"\n📈 Datensatz enthält {len(df)} Blumen")
print(f"🌸 Blumenarten: {', '.join(iris.target_names)}")

## 5. Daten visualisieren

In [ ]:
# Streudiagramm: Kelchblattlänge vs. Kelchblattbreite
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['red', 'green', 'blue']
for i, species in enumerate(iris.target_names):
    ax.scatter(X[y == i, 0], X[y == i, 1], 
                c=colors[i], label=species, alpha=0.6, s=100)

ax.set_xlabel('Kelchblattlänge (cm)', fontsize=12)
ax.set_ylabel('Kelchblattbreite (cm)', fontsize=12)
ax.set_title('Iris-Blumen: Unterschiede in den Maßen', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Die verschiedenen Blumenarten haben unterschiedliche Maße!")
print("   Das ist genau das Muster, das unsere KI lernen soll.")

## 6. Daten aufteilen: Training vs. Test

In [ ]:
# 80% Training, 20% Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"🔧 Trainingsdaten: {len(X_train)} Blumen")
print(f"🧪 Testdaten: {len(X_test)} Blumen")
print("\n💭 Warum aufteilen?")
print("   → Training: Damit lernt die KI")
print("   → Test: Damit prüfen wir, ob sie wirklich gelernt hat")

## 7. Daten normalisieren

In [ ]:
# Daten auf ähnliche Skala bringen
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("📏 Daten wurden normalisiert")
print("\n💭 Warum normalisieren?")
print("   → Alle Merkmale haben unterschiedliche Einheiten")
print("   → Normalisierung hilft der KI, besser zu lernen")
print(f"\n   Vorher (Beispiel): {X_train[0]}")
print(f"   Nachher: {X_train_scaled[0]}")

## 8. KI-Modell erstellen

In [ ]:
# Neuronales Netz mit 2 versteckten Schichten
model = MLPClassifier(
    hidden_layer_sizes=(10, 10),  # 2 Schichten mit je 10 Neuronen
    max_iter=1000,                # Maximal 1000 Trainingsrunden
    random_state=42,
    verbose=True                  # Fortschritt anzeigen
)

print("🧠 Neuronales Netz erstellt!")
print("\n📐 Architektur:")
print("   Eingabe: 4 Neuronen (für 4 Merkmale)")
print("   Versteckte Schicht 1: 10 Neuronen")
print("   Versteckte Schicht 2: 10 Neuronen")
print("   Ausgabe: 3 Neuronen (für 3 Blumenarten)")

## 9. Training starten! 🚀

In [ ]:
print("⏳ Training läuft...\n")
model.fit(X_train_scaled, y_train)
print("\n✅ Training abgeschlossen!")

## 10. KI testen

In [ ]:
# Vorhersagen auf Testdaten
y_pred = model.predict(X_test_scaled)

# Genauigkeit berechnen
accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 Genauigkeit auf Testdaten: {accuracy * 100:.1f}%")

# Einzelne Vorhersagen anzeigen
print("\n🔍 Beispiel-Vorhersagen (erste 10 Testblumen):")
print("-" * 50)
for i in range(min(10, len(y_test))):
    actual = iris.target_names[y_test[i]]
    predicted = iris.target_names[y_pred[i]]
    status = "✓" if y_test[i] == y_pred[i] else "✗"
    print(f"{status} Tatsächlich: {actual:15} | Vorhergesagt: {predicted}")

## 11. Confusion Matrix: Wo macht die KI Fehler?

In [ ]:
# Confusion Matrix erstellen und visualisieren
cm = confusion_matrix(y_test, y_pred)

# Manuelle Visualisierung (JupyterLite-kompatibel)
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, cmap='Blues')

# Achsenbeschriftungen
ax.set_xticks(np.arange(len(iris.target_names)))
ax.set_yticks(np.arange(len(iris.target_names)))
ax.set_xticklabels(iris.target_names)
ax.set_yticklabels(iris.target_names)

# Zahlen in die Zellen schreiben
for i in range(len(iris.target_names)):
    for j in range(len(iris.target_names)):
        text_color = "white" if cm[i, j] > cm.max()/2 else "black"
        text = ax.text(j, i, cm[i, j], ha="center", va="center", 
                       color=text_color, fontsize=16, fontweight='bold')

ax.set_ylabel('Tatsächliche Klasse')
ax.set_xlabel('Vorhergesagte Klasse')
ax.set_title('Confusion Matrix: Wie gut ist unsere KI?')
plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

print("\n💡 So liest du die Matrix:")
print("   → Diagonale (oben links nach unten rechts) = richtige Vorhersagen")
print("   → Andere Felder = Fehler der KI")

## 12. Eigene Vorhersage machen

In [ ]:
# Beispiel: Eine neue Blume klassifizieren
neue_blume = np.array([[5.1, 3.5, 1.4, 0.2]])  # Kelch- und Blütenblatt-Maße

# Normalisieren und vorhersagen
neue_blume_scaled = scaler.transform(neue_blume)
vorhersage = model.predict(neue_blume_scaled)
wahrscheinlichkeiten = model.predict_proba(neue_blume_scaled)

print("🌺 Neue Blume klassifizieren:")
print(f"   Maße: Kelchblatt {neue_blume[0][0]}×{neue_blume[0][1]} cm, "
      f"Blütenblatt {neue_blume[0][2]}×{neue_blume[0][3]} cm")
print(f"\n🤖 KI sagt: {iris.target_names[vorhersage[0]]}")
print("\n📊 Wahrscheinlichkeiten:")
for i, species in enumerate(iris.target_names):
    print(f"   {species}: {wahrscheinlichkeiten[0][i] * 100:.1f}%")

## 13. Experimentiere selbst!

Versuche folgendes:
- Ändere die Netzwerk-Architektur (z.B. `hidden_layer_sizes=(20, 20)` oder `(5, 5)`)
- Verändere die Anzahl der Trainingsrunden (`max_iter=500` oder `max_iter=2000`)
- Ändere die Werte in `neue_blume` und schaue, was die KI vorhersagt

### Zusammenfassung

**Was haben wir gelernt? 🎓**

1. **Daten laden**: Wir haben einen Datensatz mit Blumenmaßen geladen
2. **Daten erkunden**: Visualisierung zeigt Unterschiede zwischen Arten
3. **Daten aufteilen**: Training (80%) und Test (20%)
4. **Normalisierung**: Daten auf gleiche Skala bringen
5. **Modell erstellen**: Neuronales Netz mit 2 Schichten
6. **Training**: KI lernt Muster aus Trainingsdaten
7. **Testen**: Überprüfung auf ungesehenen Daten
8. **Vorhersagen**: KI kann neue Blumen klassifizieren

🎉 **Herzlichen Glückwunsch! Du hast deine erste KI trainiert!**